### groupby

In [2]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                    'key2' : ['one', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.492175,0.554576
1,a,two,-0.868462,-0.779343
2,b,one,1.082298,-2.071727
3,b,two,0.476077,0.185430
4,a,one,-0.615303,0.487168


In [5]:
# 按key1进行分组，计算data1的均值
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a   -0.330530
b    0.779187
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.061564
      two    -0.868462
b     one     1.082298
      two     0.476077
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.061564,-0.868462
b,1.082298,0.476077


* 在上述例子中分组键均为Series。实际上，分组键可以是任何长度适当的数组。

In [11]:
states = np.array(['Ohio', 'California', 'Ohio', 'California', 'Ohio'])
years = np.array([2005, 2005, 2006, 2006, 2005])
df['data1'].groupby([states,years]).mean().unstack()

,2005,2006
California,-0.868462,0.476077
Ohio,-0.061564,1.082298


* 通常，分组信息就位于相同的要处理DataFrame中。可以将列名（可以是字符串、数字或其他Python对象）用作分组键


In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.330530,0.087467
b,0.779187,-0.943148


在执行df.groupby('key1').mean()时，结果中没有key2列。这是因为df['key2']不是数值数据（俗称“麻烦列”），所以被从结果中排除了。默认情况下，所有数值列都会被聚合.

In [15]:
# size方法返回一个含有分组大小的Series
df.groupby(['key1','key2']).size().unstack()

key2,one,two
key1,,
a,2,1
b,1,1


#### 对分组进行迭代
GroupBy对象支持迭代，可以产生一组二元元组:由分组名和数据块组成

In [16]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.492175  0.554576
1    a  two -0.868462 -0.779343
4    a  one -0.615303  0.487168
b
  key1 key2     data1     data2
2    b  one  1.082298 -2.071727
3    b  two  0.476077  0.185430


多重键的情况，元组的第一个元素是由键值组成的元组：

In [18]:
for (k1,k2) ,group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.492175  0.554576
4    a  one -0.615303  0.487168
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.868462 -0.779343
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.082298 -2.071727
('b', 'two')
  key1 key2     data1    data2
3    b  two  0.476077  0.18543


其他可以做的操作

In [21]:
pieces = dict(list(df.groupby('key1')))

In [22]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.082298,-2.071727
3,b,two,0.476077,0.185430


* groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组

In [28]:
# 根据dtype对列进行分组
grouped = df.groupby(df.dtypes, axis=1)

In [29]:
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.492175  0.554576
1 -0.868462 -0.779343
2  1.082298 -2.071727
3  0.476077  0.185430
4 -0.615303  0.487168
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


#### 选取一列或列的子集进行操作
* 对由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。

In [38]:
# 与df['data1'].groupby(df['key1'])结果一致
df.groupby('key1')['data1'].mean()

key1
a   -0.330530
b    0.779187
Name: data1, dtype: float64

In [36]:
# 与groupby('key1')[['data2']]结果一致
df.groupby('key1')[['data1']].mean()#以DataFrame形式得到结果

,data1
key1,
a,-0.330530
b,0.779187


In [39]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.061564  0.520872
     two  -0.868462 -0.779343
b    one   1.082298 -2.071727
     two   0.476077  0.185430

#### 通过字典或序列进行分组

In [41]:
people = pd.DataFrame(np.random.randn(5, 5),
                       columns=['a', 'b', 'c', 'd', 'e'],
                       index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,-1.117216,-1.343237,0.025930,-1.582074,-0.608055
Steve,-1.142787,0.269830,-0.902049,0.316981,-2.322036
Wes,-1.338671,NaN,NaN,-1.050641,1.738420
Jim,-0.705276,0.821838,0.781866,2.035365,2.155038
Travis,-1.063899,-0.633544,-1.660406,0.524530,2.040226


In [43]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
            'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,-1.556143,-3.068508
Steve,-0.585068,-3.194993
Wes,-1.050641,0.399750
Jim,2.817232,2.271600
Travis,-1.135875,0.342783


* Series也有同样的功能，它可以被看做一个固定大小的映射

In [44]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [45]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


#### 通过函数进行分组
* 任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。

In [46]:
# 按索引（名字）字符长度分组
people.groupby(len).sum()

,a,b,c,d,e
3,-3.161162,-0.521399,0.807797,-0.597349,3.285403
5,-1.142787,0.269830,-0.902049,0.316981,-2.322036
6,-1.063899,-0.633544,-1.660406,0.524530,2.040226


In [47]:
people.groupby(len).min()

,a,b,c,d,e
3,-1.338671,-1.343237,0.025930,-1.582074,-0.608055
5,-1.142787,0.269830,-0.902049,0.316981,-2.322036
6,-1.063899,-0.633544,-1.660406,0.524530,2.040226


#### 根据索引级别分组

In [49]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.053066 -0.451119 -0.833667  0.279589  0.368862
1     -0.647467  1.307029 -2.148584  0.974965  0.118486
2      0.519731  0.729301 -0.326813 -0.238345 -1.690266
3      0.981877  0.385952 -1.063351  3.402603  0.097306

In [50]:
# 要根据级别分组，使用level关键字传递级别序号或名字
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 数据聚合

In [51]:
df

,key1,key2,data1,data2
0,a,one,0.492175,0.554576
1,a,two,-0.868462,-0.779343
2,b,one,1.082298,-2.071727
3,b,two,0.476077,0.185430
4,a,one,-0.615303,0.487168


In [75]:
# 样本分位数
print(df['data1'].quantile(0.9))
print(df['data1'].quantile(1)-(df['data1'].quantile(1)-df['data1'].quantile(0.75))*0.4)
df['data1'].quantile(0.75)+(df['data1'].quantile(1)-df['data1'].quantile(0.75))*0.6

0.8462488018725803
0.8462488018725803


0.8462488018725802

* 调用自己定义的聚合函数，只需将其传入aggregate或agg方法即可。

In [78]:
def p_to_p (arr):
    return arr.max()-arr.min()
grouped = df.groupby('key1')
grouped.agg(p_to_p)

,data1,data2
key1,,
a,1.360637,1.333919
b,0.606222,2.257157


In [81]:
# 非聚合运算在这里也可以使用
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.330530  0.723640 -0.868462 -0.741883 -0.615303 -0.061564   
b      2.0  0.779187  0.428663  0.476077  0.627632  0.779187  0.930743   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.492175   3.0  0.087467  0.751436 -0.779343 -0.146087  0.487168   
b     1.082298   2.0 -0.943148  1.596051 -2.071727 -1.507437 -0.943148   

                          
           75%       max  
key1                      
a     0.520872  0.554576  
b    -0.378859  0.185430

#### 面向列的多函数应用

In [82]:
tips = pd.read_csv(r'E:\ML\pydata_2nd_edition\pydata-book-2nd-edition\examples\tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [83]:
grouped = tips.groupby(['day','smoker'])

In [85]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean').unstack()

smoker,No,Yes
day,,
Fri,0.151650,0.174783
Sat,0.158048,0.147906
Sun,0.160113,0.187250
Thur,0.160298,0.163863


In [88]:
# 如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名
grouped_pct.agg(['mean','std',p_to_p])

mean       std    p_to_p
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [89]:
# 如果要对函数运算结果给出自定义名称，可以传入一个由(name,function)元组组成的列表，则各元组的第一个元素就会被用作DataFrame的列名
grouped_pct.agg([('均值','mean'),('标准差','std')])

均值       标准差
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [91]:
functions = ['count','min','max']
result = grouped['tip_pct','total_bill'].agg(functions)
result

tip_pct                     total_bill              
              count       min       max      count    min    max
day  smoker                                                     
Fri  No           4  0.120385  0.187735          4  12.46  22.75
     Yes         15  0.103555  0.263480         15   5.75  40.17
Sat  No          45  0.056797  0.291990         45   7.25  48.33
     Yes         42  0.035638  0.325733         42   3.07  50.81
Sun  No          57  0.059447  0.252672         57   8.77  48.17
     Yes         19  0.065660  0.710345         19   7.25  45.35
Thur No          45  0.072961  0.266312         45   7.51  41.19
     Yes         17  0.090014  0.241255         17  10.34  43.11

从上述可见，结果DataFrame拥有层次化的列，这相当于分别对各列进行聚合，然后用concat将结果组装到一起，使用列名用作keys参数

In [92]:
# 如果要对一个列或不同的列应用不同的函数。具体的办法是向agg传入一个从列名映射到函数的字典
grouped.agg({'tip':'max','size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [93]:
grouped.agg({'tip':['max','min'],'size':'sum'})

tip       size
               max   min  sum
day  smoker                  
Fri  No       3.50  1.50    9
     Yes      4.73  1.00   31
Sat  No       9.00  1.00  115
     Yes     10.00  1.00  104
Sun  No       6.00  1.01  167
     Yes      6.50  1.50   49
Thur No       6.70  1.25  112
     Yes      5.00  2.00   40

#### 以“没有行索引”的形式返回聚合数据
* 上述示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层次化的）。由于并不总是需要如此，可以向groupby传入as_index=False以禁用该功能

In [97]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


* 对结果调用reset_index也能得到这种形式的结果,但使用as_index=False方法可以避免一些不必要的计算。

In [96]:
tips.groupby(['day','smoker']).mean().reset_index()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### apply：一般性的“拆分－应用－合并”

In [99]:
tips.groupby('smoker').apply(np.max)

,total_bill,tip,smoker,day,time,size,tip_pct
smoker,,,,,,,
No,48.33,9.0,No,Thur,Lunch,6,0.291990
Yes,50.81,10.0,Yes,Thur,Lunch,5,0.710345


In [100]:
tips.groupby('smoker').agg('max')

,total_bill,tip,day,time,size,tip_pct
smoker,,,,,,
No,48.33,9.0,Thur,Lunch,6,0.291990
Yes,50.81,10.0,Thur,Lunch,5,0.710345


In [101]:
tips.groupby('smoker').max()

,total_bill,tip,day,time,size,tip_pct
smoker,,,,,,
No,48.33,9.0,Thur,Lunch,6,0.291990
Yes,50.81,10.0,Thur,Lunch,5,0.710345


In [102]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [103]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [104]:
tips.groupby('smoker').agg(top)

ValueError: Shape of passed values is (7, 2), indices imply (6, 2)

* 对于聚合函数，apply和agg没有大的差别。agg方法仅能随分组数据进行聚合操作,调用自定义的非聚合函数报错，而apply能够对自定义的非聚合的函数进行调用，具有普适性

In [105]:
tips.groupby('smoker')['tip_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


在GroupBy中，调用诸如describe之类的方法时，实际上只是应用了下面两条代码的快捷方式:
<p>f = lambda x: x.describe()
<p>grouped.apply(f)

#### 禁止分组键
从上面的例子中可以看出，分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将group_keys=False传入groupby即可禁止该效果

In [109]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


#### 分位数和桶分析
- pandas有一些能根据指定面元或样本分位数将数据离散化的工具（比如cut和qcut）。将这些函数跟groupby结合起来，就能实现对数据集的桶（bucket）或分位数（quantile）分析。

In [115]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                       'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, bins = 4)

In [116]:
quartiles[:10]

0     (0.3, 1.944]
1    (-1.344, 0.3]
2    (-1.344, 0.3]
3     (0.3, 1.944]
4    (-1.344, 0.3]
5    (-1.344, 0.3]
6    (-1.344, 0.3]
7    (-1.344, 0.3]
8    (-1.344, 0.3]
9    (-1.344, 0.3]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.994, -1.344] < (-1.344, 0.3] < (0.3, 1.944] < (1.944, 3.589]]

In [118]:
# 由cut返回的Categorical对象可直接传递到groupby
def get_stat(group):
    return {'min': group.min(), 'max': group.max(),
             'count': group.count(), 'mean': group.mean()}
frame.data2.groupby(quartiles).apply(get_stat).unstack()

,count,max,mean,min
data1,,,,
"(-2.994, -1.344]",91.0,2.122605,-0.033138,-2.027631
"(-1.344, 0.3]",550.0,2.754354,0.013045,-4.011254
"(0.3, 1.944]",334.0,2.592350,-0.020484,-2.991548
"(1.944, 3.589]",25.0,2.148002,0.007598,-2.072023


* 以上是长度相同的区间，要根据样本分位数得到大小相等的区间，使用qcut。传入labels=False即可只获取分位数的编号

In [122]:
group1 = pd.qcut(frame.data1, 10)
grouped = frame.data2.groupby(group1)
grouped.apply(get_stat).unstack()

,count,max,mean,min
data1,,,,
"(-2.989, -1.303]",100.0,2.122605,-0.070872,-2.027631
"(-1.303, -0.826]",100.0,2.588849,-0.032528,-4.011254
"(-0.826, -0.522]",100.0,2.754354,0.135086,-2.477737
"(-0.522, -0.284]",100.0,1.714391,-0.026061,-2.492055
"(-0.284, -0.0488]",100.0,2.695049,0.140190,-1.911813
"(-0.0488, 0.208]",100.0,2.562858,-0.180950,-2.221438
"(0.208, 0.455]",100.0,2.592350,0.085566,-2.098185
"(0.455, 0.79]",100.0,1.979130,-0.057759,-2.991548
"(0.79, 1.221]",100.0,2.209656,0.062649,-2.033772


In [121]:
group1 = pd.qcut(frame.data1, 10,labels=False)
grouped = frame.data2.groupby(group1)
grouped.apply(get_stat).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.122605,-0.070872,-2.027631
1,100.0,2.588849,-0.032528,-4.011254
2,100.0,2.754354,0.135086,-2.477737
3,100.0,1.714391,-0.026061,-2.492055
4,100.0,2.695049,0.140190,-1.911813
5,100.0,2.562858,-0.180950,-2.221438
6,100.0,2.592350,0.085566,-2.098185
7,100.0,1.979130,-0.057759,-2.991548
8,100.0,2.209656,0.062649,-2.033772


### 示例

#### 用特定分组值填充缺失值

In [124]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.012064
2         NaN
3    1.104002
4         NaN
5    0.233884
dtype: float64

In [125]:
# 用平均值填充NA值
s.fillna(s.mean())

0    0.441941
1   -0.012064
2    0.441941
3    1.104002
4    0.441941
5    0.233884
dtype: float64

In [126]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
           'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.391512
New York     -0.569929
Vermont       0.984902
Florida      -1.358941
Oregon       -0.794661
Nevada        0.833541
California    0.198742
Idaho        -0.709798
dtype: float64

In [128]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.391512
New York     -0.569929
Vermont            NaN
Florida      -1.358941
Oregon       -0.794661
Nevada             NaN
California    0.198742
Idaho              NaN
dtype: float64

In [129]:
data.groupby(group_key).mean()

East   -1.106794
West   -0.297959
dtype: float64

In [131]:
# 用分组平均值去填充NA值
fillna_mean = lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fillna_mean)

Ohio         -1.391512
New York     -0.569929
Vermont      -1.106794
Florida      -1.358941
Oregon       -0.794661
Nevada       -0.297959
California    0.198742
Idaho        -0.297959
dtype: float64

In [134]:
# 可以在代码中预定义各组的填充值
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g:g.fillna(fill_values[g.name])#g.name 索引名称
data.groupby(group_key).apply(fill_func)

Ohio         -1.391512
New York     -0.569929
Vermont       0.500000
Florida      -1.358941
Oregon       -0.794661
Nevada       -1.000000
California    0.198742
Idaho        -1.000000
dtype: float64

#### 随机采样和排列

In [143]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck[:6]#deck为长度为52的Series，其索引包括牌名，值则是游戏中用于计分的点数

AH    1
2H    2
3H    3
4H    4
5H    5
6H    6
dtype: int64

In [138]:
# 从整副牌中抽出5张
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

6H     6
6C     6
9S     9
JD    10
JC    10
dtype: int64

In [140]:
# 从每种花色中随机抽取两张牌,花色是牌名最后一个字符
get_suits = lambda card:card[-1]# last letter is suit
deck.groupby(get_suits).apply(draw,n=2)


C  KC     10
   3C      3
D  6D      6
   5D      5
H  AH      1
   5H      5
S  7S      7
   10S    10
dtype: int64

In [141]:
deck.groupby(get_suits,group_keys=False).apply(draw,n=2)

2C     2
QC    10
AD     1
4D     4
JH    10
7H     7
6S     6
5S     5
dtype: int64

#### 分组加权平均数和相关系数

In [144]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                 'b', 'b', 'b', 'b'],
                    'data': np.random.randn(8),
                    'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-1.080608,0.710154
1,a,0.696126,0.952276
2,a,0.442613,0.531832
3,a,-0.112913,0.782945
4,b,-0.967536,0.663874
5,b,0.350083,0.952827
6,b,-0.140106,0.204406
7,b,0.456036,0.153083


In [145]:
# 利用category计算分组加权平均数
grouped = df.groupby('category')
get_avg = lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_avg)

category
a    0.014274
b   -0.135539
dtype: float64

In [150]:
close_px = pd.read_csv(r'E:\ML\pydata_2nd_edition\pydata-book-2nd-edition\examples\stock_px_2.csv',
                       parse_dates=True,
                        index_col=0)
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [151]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [155]:
# 每列和SPX列的成对相关系数
spx_corr = lambda x:x.corrwith(x['SPX'])
# 使用pct_change计算close_px的百分比变化
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [156]:
get_year = lambda x:x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [157]:
# 计算Apple和Microsoft的年相关系数
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

#### 组级别的线性回归
* 利用statsmodels计量经济学库对各数据块执行最小二乘法回归

In [163]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    result = sm.OLS(Y,X).fit()
    return result.params,result.rsquared #返回回归参数,R平方
# 按年计算AAPL对SPX收益率的线性回归
by_year.apply(regress,'AAPL', ['SPX'])

2003     ([1.2008408960866115], 0.2957988753960885)
2004    ([1.3949954509222073], 0.14218064938476072)
2005     ([1.777145311377742], 0.21692978232501603)
2006    ([1.6465478741394204], 0.18465543339011237)
2007     ([1.2051863495964883], 0.2549624429333759)
2008     ([0.9706595444204551], 0.4662267262356765)
2009     ([0.8889442226286153], 0.4969971489798496)
2010     ([1.0579030319812193], 0.5045141136089378)
2011     ([0.8063518145533513], 0.4746039582993896)
dtype: object

### 透视表和交叉表
#### 透视表
* DataFrame有一个pivot_table方法，此外还有一个pandas.pivot_table函数。除能为groupby提供便利之外，pivot_table还可以添加分项小计，也叫做margins。

In [167]:
# 根据day和smoker计算分组消费平均数,pivot_table的默认聚合类型
tips.pivot_table(index=['day','smoker'])#与tips.groupby(['day','smoker']).agg('mean')结果一致

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [168]:
# 对'tip_pct','size'进行聚合
# 按time分组，day放在行，smoker放在列
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [169]:
# 传入margins=True添加分项小计
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

All值为平均数
<p>All列:不单独考虑烟民与非烟民，All行:不单独考虑行分组两个级别中的任何单项

In [211]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

对输出结果中的NA值进行填充

In [216]:
# 使用len可以得到有关分组大小的计数或频率
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,fill_value=0)

day            Fri  Sat  Sun  Thur
time   smoker                     
Dinner No        3   45   57     1
       Yes       9   42   19     0
Lunch  No        1    0    0    44
       Yes       6    0    0    17

#### 交叉表
* 交叉表（cross-tabulation，简称crosstab）是一种用于计算分组频率的特殊透视表

In [242]:
data = pd.DataFrame({'sample':np.arange(1,11),
                   'nationality':['USA','Japan','USA']*3+['Japan'],
                   'handedness':['Right-handed','Left-handed']*4+['Left-handed']*2})
data

,sample,nationality,handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,USA,Left-handed
4,5,Japan,Right-handed
5,6,USA,Left-handed
6,7,USA,Right-handed
7,8,Japan,Left-handed
8,9,USA,Left-handed
9,10,Japan,Left-handed


In [243]:
# 根据国籍和用手习惯对这段数据进行统计汇总
pd.crosstab(data.nationality,data.handedness,margins=True)
# 或data.pivot_table('sample',index='nationality',columns='handedness',aggfunc=len,margins=True)

handedness,Left-handed,Right-handed,All
nationality,,,
Japan,3,1,4
USA,3,3,6
All,6,4,10


In [255]:
# 按time，smoker，day进行统计计数
pd.crosstab([tips.time,tips.smoker],tips.day,margins=True)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244